# Big Bird Classifier for all

In [ ]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install torch --quiet
!pip install sentencepiece --quiet
!pip install --upgrade accelerate --quiet

In [ ]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import warnings
import os

warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/raw

/content/drive/.shortcut-targets-by-id/1j0RASnE3IntTbVJIRuwXAvt885-X9BEv/Proyecto DL + NLP/Entrega_Final/data/raw


In [ ]:
%ls

ADHD/     BPD/                   DATASET_CONTROL/  PTSD/
Anxiety/  combined_balanced.csv  Depression/       Schizophrenia/
Bipolar/  combined.csv           EDA/


In [ ]:
data = pd.read_csv('./combined_balanced.csv')

In [ ]:
data

,subreddit,post,label
0,control,What do you do when most of a class passes and...,0
1,control,"Picking a chapter book for ""cool down time"" I'...",0
2,control,How can I prevent alumni from visiting me? As ...,0
3,control,Why I feel like a dumb student In elementary S...,0
4,control,Anonymous questionnaire on Social Anxiety Hell...,0
...,...,...,...
143995,bipolarreddit,Business travel w/ bipolar? Hi Friends -\n\nI'...,3
143996,bipolarreddit,Finding a job when depressed is impossible. Fi...,3
143997,bipolarreddit,Mania?? I was diagnosed with bipolar over 4 ye...,3
143998,bipolarreddit,Are any of you successfully managing your cond...,3


In [ ]:
data['subreddit'].unique()

array(['control', 'depression', 'anxiety', 'adhd', 'bpd', 'EDAnonymous',
       'schizophrenia', 'ptsd', 'bipolarreddit'], dtype=object)

In [ ]:
data['subreddit'] = data['subreddit'].str.replace('EDAnonymous','eda')
data['subreddit'] = data['subreddit'].str.replace('bipolarreddit','bipolar')

In [ ]:
data['subreddit'].unique()

array(['control', 'depression', 'anxiety', 'adhd', 'bpd', 'eda',
       'schizophrenia', 'ptsd', 'bipolar'], dtype=object)

In [ ]:
train_text_df  = data.rename(columns={'subreddit': 'entities', 'post': 'text', 'label': 'labels'})

In [ ]:
train_text_df ["id"] = train_text_df .index + 1

In [ ]:
train_text_df ['entities'] = train_text_df ['entities'].str.strip('()').str.split(',')

In [ ]:
from sklearn.utils import shuffle
train_text_df = shuffle(train_text_df)

In [ ]:
train_text_df.drop(['labels'], axis=1, inplace=True)

In [ ]:
train_text_df = train_text_df.reset_index(drop=True)

In [ ]:
train_text_df 

,entities,text,id
0,[depression],How do I hide my depression? It's making my da...,18455
1,[ptsd],Feeling the need to reach out to abuser during...,125564
2,[bipolar],Inspired by the other post: lamictal solo trea...,133794
3,[anxiety],First anxiety attack Always struggled with anx...,47085
4,[bipolar],Here comes the mania! It's nearly 1am. I have ...,129229
...,...,...,...
143995,[ptsd],Ocular migraine. The fuck. Y’all doing this to...,127576
143996,[bipolar],Going through divorce and I don't care is this...,142672
143997,[eda],Peach is reaching out for someone to adopt and...,86518
143998,[adhd],First thing in the morning Do any of you lovel...,48313


In [ ]:
import ast

train_text_df['entities'] = train_text_df['entities'].apply(lambda x:ast.literal_eval(str(x)))

print(train_text_df['entities'].values[0])

['depression']


In [ ]:
train_text_df = train_text_df[:]

In [1]:
import datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import BigBirdForTokenClassification, BigBirdTokenizerFast
from torch import cuda
import torch

ModuleNotFoundError: ignored

In [ ]:
config = {'model_name': 'google/bigbird-roberta-base',
         'max_length': 150,
         'train_batch_size':4,
         'valid_batch_size':4,
         'epochs':5,
         'learning_rate':5e-05,
         'max_grad_norm':10,
          'warmup':0.1,
          "grad_acc":8,
          "model_save_path":"big-bird",
         'device': 'cuda' if cuda.is_available() else 'cpu'}

output_labels = ['control', 'depression', 'anxiety', 'adhd', 'bpd', 'eda',
       'schizophrenia', 'ptsd', 'bipolar']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [ ]:
train_text_df['labels'] = train_text_df['entities'].apply(lambda x: [labels_to_ids[i] for i in x])

In [ ]:
train_text_df.head()

,entities,text,id,labels
0,[depression],How do I hide my depression? It's making my da...,18455,[1]
1,[ptsd],Feeling the need to reach out to abuser during...,125564,[7]
2,[bipolar],Inspired by the other post: lamictal solo trea...,133794,[8]
3,[anxiety],First anxiety attack Always struggled with anx...,47085,[2]
4,[bipolar],Here comes the mania! It's nearly 1am. I have ...,129229,[8]


In [ ]:
tokenizer = BigBirdTokenizerFast.from_pretrained(config['model_name'])
model = BigBirdForTokenClassification.from_pretrained(config['model_name'],
                                                     num_labels=len(output_labels))

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

In [ ]:
converted = tokenizer(train_text_df.loc[0].values[1].split(),
                      is_split_into_words=True, return_offsets_mapping=True)

In [ ]:
ix = 0
for i,j in zip(tokenizer.convert_ids_to_tokens(converted['input_ids']), converted['offset_mapping']):
    print(i, j)
    ix += 1
    if ix == 15:
        break

[CLS] (0, 0)
▁How (0, 3)
▁do (0, 2)
▁I (0, 1)
▁hide (0, 4)
▁my (0, 2)
▁depression (0, 10)
? (10, 11)
▁It (0, 2)
's (2, 4)
▁making (0, 6)
▁my (0, 2)
▁dad (0, 3)
▁loose (0, 5)
▁his (0, 3)


In [ ]:
def tokenizer_data(example):
    encoding = tokenizer(example['text'].split(),
                         is_split_into_words=True,
                         truncation=True,
                         padding='max_length', 
                         return_offsets_mapping=True,
                         max_length=config['max_length'])
    i = 0
    labels = example['labels']
    encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
    for idx, mapping in enumerate(encoding["offset_mapping"]):
        if mapping[0] == 0 and mapping[1] != 0:
            try:
                encoded_labels[idx] = labels[i]
            except:
                pass
            i += 1
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
    item['labels'] = torch.as_tensor(encoded_labels)
    return item

In [ ]:
dataset = datasets.Dataset.from_pandas(train_text_df)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 129600
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 14400
    })
})

In [ ]:
text = dataset['train'][1]

In [ ]:
converted = tokenizer_data(text)

converted

{'input_ids': tensor([   65, 18731,  3799,  2852,   477, 11363,  5458, 19370, 18595,  1159,
           116,   321,   100, 48515,   578,   415,   100,   404,   688,  1905,
           419,  2212,   385,  4584,   618,   112,   391,  6552,   415,  1355,
           624,  2014,   391,  3150,   112,   391,   508,   689,   415,   100,
           177,  1117,   385,  9908,   494, 11596,   388,  2164,  1242,   717,
         27298, 42355,   114,  2430,  2328,   385, 10749,   391,  3859,   718,
          1527, 11466,   385,   446,   578,   112,  6512,  4304,   529,  2158,
           113,   372,   113, 25903,  1618,  2010,   114,   871,  3232,  2669,
           115, 41503,   419,   365,  2339,  3722, 50348,   126,   136, 20780,
            66,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
converted

i=0
for token, label in zip(tokenizer.convert_ids_to_tokens(converted["input_ids"]), 
                        converted["labels"]):
    print(token, label, converted['offset_mapping'][i])
    i+=1
    if i == 15:
        break

[CLS] tensor(-100) tensor([0, 0])
▁FE tensor(5) tensor([0, 2])
EL tensor(-100) tensor([2, 4])
ING tensor(-100) tensor([4, 7])
▁F tensor(-100) tensor([0, 1])
AST tensor(-100) tensor([1, 4])
▁AND tensor(-100) tensor([0, 3])
▁STR tensor(-100) tensor([0, 3])
ONG tensor(-100) tensor([3, 6])
▁: tensor(-100) tensor([0, 1])
0 tensor(-100) tensor([1, 2])
▁ tensor(-100) tensor([0, 1])
<unk> tensor(-100) tensor([0, 2])
▁Literally tensor(-100) tensor([0, 9])
▁all tensor(-100) tensor([0, 3])


In [ ]:
dataset = dataset.map(tokenizer_data)

dataset

Map:   0%|          | 0/129600 [00:00<?, ? examples/s]

Map:   0%|          | 0/14400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 129600
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 14400
    })
})

In [ ]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',
                                         'labels'])

dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 129600
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 14400
    })
})

In [ ]:
for a in dataset['train']:
    if a['input_ids'].shape[0] != a['attention_mask'].shape[0]:
        print(a)
        break

In [ ]:
trainer_args = TrainingArguments(report_to='none',
                                output_dir="../../output/classificator/BigBird/All",
                                num_train_epochs=config['epochs'],
                                evaluation_strategy ='epoch',
                                per_device_train_batch_size=config['train_batch_size'],
                                per_device_eval_batch_size=config['valid_batch_size'],
                                fp16=True,
                                save_strategy = "epoch",
                                warmup_ratio= config['warmup'],
                                gradient_accumulation_steps=config['grad_acc'],
                                logging_strategy="epoch",
                                save_total_limit=1
                                )

trainer = Trainer(model=model,
                  args=trainer_args, 
                  train_dataset = dataset['train'],
                  eval_dataset=dataset['test'],
                  tokenizer=tokenizer)

In [ ]:
#trainer.train()

You're using a BigBirdTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 150 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss
1,0.738400,0.486419


Epoch,Training Loss,Validation Loss
1,0.738400,0.486419
2,0.405300,0.408280


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1940 in _inner_training_loop     │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2745 in training_step            │
│                                                                                                  │
│   2742 │   │   │   loss = loss / self.args.gradient_accumulation_steps                           │
│   2743 │   │                                                                                     │
│   2744 │   │   if self.do_grad_scaling:                                                          │
│ ❱ 2745 │   │   │   self.scaler.scale(loss).backward()                                            │
│   2746 │   │   elif self.use_apex:                                                               │
│   2747 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2748 │   │   │   │   scaled_loss.backward()                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_gra

In [ ]:
trainer.train("../../output/classificator/BigBird/All/checkpoint-16200")

Epoch,Training Loss,Validation Loss
5,0.072700,0.440286


TrainOutput(global_step=20250, training_loss=0.014547444661458334, metrics={'train_runtime': 2470.9977, 'train_samples_per_second': 262.242, 'train_steps_per_second': 8.195, 'total_flos': 5.0922897855786e+16, 'train_loss': 0.014547444661458334, 'epoch': 5.0})

In [ ]:
device = config['device']

In [ ]:
trainer.model.eval()
def inference(sentence):
    inputs = tokenizer(sentence.split(),
                        is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=200,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = trainer.model(input_ids=ids, attention_mask=mask, return_dict=False)
#     print(outputs)
    logits = outputs[0]
    
    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
    print(logits.shape, active_logits.shape, flattened_predictions.shape)
    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [ ]:
#Anxiety
text_1= 'Ive never taken it to a doctor, but Ive heard that it could help for this problem, but I dont want to take it to the point where I am constantly afraid of dying.'
pred_1, _ = inference(text_1)
print(pred_1[0])

torch.Size([1, 200, 9]) torch.Size([200, 9]) torch.Size([200])
anxiety


In [ ]:
#ADHD
text_2= 'I am not sure if this is a sign that I am not sick or if I am just lazy and have no sense of direction.'
pred_2, _ = inference(text_2)
print(pred_2[0])

torch.Size([1, 200, 9]) torch.Size([200, 9]) torch.Size([200])
schizophrenia


In [ ]:
model = MyModelDefinition(args)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


checkpoint = torch.load('load/from/path/model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
